In [ ]:
storageAccount="agdbtrngaadass2storage"
mountpoint = "/mnt/Gen2Source"
storageEndPoint ="abfss://rawdata@{}.dfs.core.windows.net/".format(storageAccount)
print ('Mount Point ='+mountpoint)

#ClientId, TenantId and Secret is for the Application(ADLSGen2App) was have created as part of this recipe
clientID ="xxxxx69-xx6e-4xxb-b6fa-a9sdad221210db"
tenantID =""
clientSecret =""
oauth2Endpoint = "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)


configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": clientID,
           "fs.azure.account.oauth2.client.secret": clientSecret,
           "fs.azure.account.oauth2.client.endpoint": oauth2Endpoint}

try:
  dbutils.fs.mount(
  source = storageEndPoint,
  mount_point = mountpoint,
  extra_configs = configs)
except:
    print("Already mounted...."+mountpoint)


In [ ]:
display(dbutils.fs.ls("/mnt/Gen2Source/Customer/parquetFiles"))

In [ ]:
db = "tpchdb"
 
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")
 
spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

In [ ]:
import random
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
  

### Reading Customer parquet files from the mount point and writing to Delta tables.

In [ ]:
#Reading parquet files and adding a new column to the dataframe and writing to delta table
cust_path = "/mnt/Gen2Source/Customer/parquetFiles"
 
df_cust = (spark.read.format("parquet").load(cust_path)
      .withColumn("timestamp", current_timestamp()))
 
df_cust.write.format("delta").mode("overwrite").save("/mnt/Gen2Source/Customer/delta")

In [ ]:
%sql
-- Creating Delta table 
DROP TABLE IF EXISTS Customer;
CREATE TABLE Customer
USING delta
location "/mnt/Gen2Source/Customer/delta"

In [ ]:
%sql
describe formatted Customer

In [ ]:
%sql
select * from Customer limit 10;

In [ ]:
%fs ls /mnt/Gen2Source/Orders/parquetFiles/

In [ ]:
#Reading Orders parquet files and adding a new column to the dataframe and writing as delta format. Creating partition on year column.
ord_path = "/mnt/Gen2Source/Orders/parquetFiles"
 
df_ord = (spark.read.format("parquet").load(ord_path)
      .withColumn("timestamp", current_timestamp())
      .withColumn("O_OrderDateYear", year(col("O_OrderDate")))
     )

# df.printSchema()
 
df_ord.write.format("delta").partitionBy("O_OrderDateYear").mode("overwrite").save("/mnt/Gen2Source/Orders/delta")

In [ ]:
%sql
DROP TABLE IF EXISTS Orders;
CREATE TABLE Orders
USING delta
location "/mnt/Gen2Source/Orders/delta"

In [ ]:
%sql
describe formatted Orders

In [ ]:
%sql
-- Getting total customer based on priority and total account balance by joing the delta tables
SELECT o.O_ORDERPRIORITY,count(o.O_CustKey) As TotalCustomer,Sum(c.C_AcctBal) As CustAcctBal
FROM Orders o 
INNER JOIN Customer c on o.O_CustKey=c.C_CustKey 
WHERE o.O_OrderDateYear>1997
GROUP BY o.O_OrderPriority
ORDER BY TotalCustomer DESC;



In [ ]:
# We can read the Delta files in Python using DeltaTable libraries as shown below 
deltaTable = DeltaTable.forPath(spark, "/mnt/Gen2Source/Orders/delta/")

In [ ]:
#Converting to DF and viewing dataframe contents
dt=deltaTable.toDF()
dt.show()

### DML Support in Delta Table.
  We can perform DML opreations like insert,delete and updates to the Delta table.

In [ ]:
# Reading new data which is in the folder newParquetFiles in the same orders folder in the /nnt/Gen2Source
# We will see how to perform merge to insert anf update the records in the delta table.

#Reading Orders parquet files and adding a new column to the dataframe and writing as delta format. Creating partition on year column.
new_ord_path = "/mnt/Gen2Source/Orders/newParquetFiles"
 
df_new_order = (spark.read.format("parquet").load(new_ord_path)
      .withColumn("timestamp", current_timestamp())
      .withColumn("O_OrderDateYear", year(col("O_OrderDate")))
     )

# df.printSchema()
 
df_new_order.write.format("delta").partitionBy("O_OrderDateYear").mode("overwrite").save("/mnt/Gen2Source/Orders/dailydata_delta")



In [ ]:
%sql
DROP TABLE IF EXISTS Orders_Daily;
CREATE TABLE Orders_Daily
USING delta
location "/mnt/Gen2Source/Orders/dailydata_delta"

In [ ]:
%sql
-- Merging into Orders from Orders_Daily table. If records matche on OrderKey then perform update eles insert
MERGE INTO Orders AS o
USING Orders_daily AS d
ON o.O_ORDERKEY = D.O_ORDERKEY
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *;

In [ ]:
%sql
-- Deleting from Delta table
DELETE FROM Orders WHERE O_ORDERKEY=1359427

In [ ]:
%sql
SELECT * FROM ORDERS WHERE O_ORDERKEY=1359427

In [ ]:
%fs ls /mnt/Gen2Source/Orders/ParquetOutputFormatted

In [ ]:
%sql CONVERT TO DELTA parquet.`dbfs:/mnt/Gen2Source/Orders/ParquetOutputFormatted`

In [ ]:
%sql
CREATE TABLE Orders_Test
USING DELTA
LOCATION "/mnt/Gen2Source/Orders/ParquetOutputFormatted"

In [ ]:
%sql
describe formatted Orders_Test

In [ ]:
#dbutils.fs.unmount("/mnt/Gen2Source")